In [ ]:
from io import StringIO
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import glob
from scipy.integrate import cumtrapz
from scipy.fft import fft
import numpy as np
from obspy import Trace
from obspy.signal.trigger import classic_sta_lta, trigger_onset


def sta_lta(df, sta=500, lta=5000, thr_on=3, thr_off=1):
    '''
    Dectect and mark trigger on and trigger off times of seismic anomalies using Short Term Average/Long Term Average algorithm

    Parameters:
    - df (pd.DataFrame): DataFrame containing the seismic data.
    - sta (int): Short Time Average window length (in samples).
    - lta (int): Long Time Average window length (in samples).
    - thr_on (float): Trigger on threshold (earthquake detected) in rel time.
    - thr_off (float): Trigger off threshold in rel time.

    Returns:
    - list: A list of tuples containing trigger on and off indices of df.
    '''
    samp_rate = 1 / (df["time_rel(sec)"][1] - df["time_rel(sec)"][0])
    offset = int(samp_rate * 240)

    cft = classic_sta_lta(df["velocity(m/s)"], int(sta * samp_rate), int(lta * samp_rate))
    on_off = np.array(trigger_onset(cft, thr_on, thr_off))

    trig_indices = []
    for i in np.arange(0, len(on_off)):
        trigger = on_off[i]
        trig_indices.append((trigger[0] - offset, trigger[1] - offset))

    return trig_indices



